In [ ]:
# Names

# Final Project: 2016 Voter Survey Data

In this data set we will be examining survey data from the 2016 presidential election to correlations between voter concerns and voter behaviors. We will be examining how priorities of voters inform who they will voter for during an election.

In [1]:
import warnings
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

In [2]:
# importing data

In [52]:
data = pd.read_csv("voter_survey_2016_data.csv", engine='python')
data.head(10)
label, data_x = dataPrep(data)

# Feature Selection
We want to examine which issues matter to which party's voters. To examine this we will be looking at columns
imiss_a_2016 - imiss_y_2016. These columns represent the surveyer's answers to questions where they were asked to rank how important a public issue was on a categorical scale: "Very important", "Somewhat important, "Not very important", "unimportant". We will be focus our data set on these columns so that our machine learning algorithm focuses purely on voter interest since that is what we're interested in investigating. We will also be including the column of who the surveyer voted for president as this will form the basis of our labels (presvote16post_2016). 

# Data Cleaning

We want to turn the data into something that we can easily use to train our models. We change our labels to: Hillary Clinton to 1, Donald Trump to 2, and all others are 0. We also transform the 23 question survey using One Hot Encoding because all of the responces are catigorical. I have also added results of questions about issues to the data set. The
overall size of the dataset is about 8000 x 46, which will expand to 8000 x 173 with one hot encoding.


In [48]:

def dataPrep(data):
    
    columns = ['presvote16post_2016', 'imiss_a_2016', 'imiss_b_2016', 'imiss_c_2016', 'imiss_d_2016', 'imiss_e_2016', 'imiss_f_2016', 'imiss_g_2016', 'imiss_h_2016', 'imiss_i_2016', 'imiss_j_2016', 'imiss_k_2016', 'imiss_l_2016', 'imiss_m_2016', 'imiss_n_2016', 'imiss_o_2016','imiss_p_2016', 'imiss_q_2016','imiss_r_2016','imiss_s_2016', 'imiss_t_2016','imiss_u_2016','imiss_x_2016','imiss_y_2016', 'immi_contribution_2016', 'immi_naturalize_2016', 'immi_makedifficult_2016', 'immi_muslim_2016', 'abortview3_2016', 'gaymar_2016', 'view_transgender_2016', 'deathpen_2016', 'deathpenfreq_2016', 'police_threat_2016', 'univhealthcov_2016', 'healthreformbill_2016', 'envwarm_2016', 'envpoll2_2016', 'affirmact_gen_2016', 'taxdoug_2016', 'govt_reg_2016', 'gvmt_involment_2016', 'tradepolicy_2016', 'free_trade_1_2016', 'free_trade_2_2016', 'free_trade_3_2016', 'free_trade_4_2016']
    df = pd.DataFrame(data=data, columns=columns)
    
    df.dropna(axis=0, inplace=True) #This drops all nans 
    labels = df['presvote16post_2016']
    df = df.drop(['presvote16post_2016'], axis=1)
    #df_clean = df.apply(lambda x: x.fillna(x.value_counts().index[0])) #this fills with mode of column
    columns = ['Crime', 'The Economy', 'Immigration', 'The Enviroment', 'Religious Liberty', 'Terrorism', 'Gay Rights', 'Education', 'Family and Medical Leave', 'Healthcare', 'Money in Politics', 'Climate Change', 'Social Security', 'Infrastructure Investment', 'Jobs', 'The Budget Deficit', 'Poverty', 'Taxes', 'Medicare', 'Abortion', 'Size of Goverment', 'Race Equality', 'Gender Equality', 'immi_contribution_2016', 'immi_naturalize_2016', 'immi_makedifficult_2016', 'immi_muslim_2016', 'abortview3_2016', 'gaymar_2016', 'view_transgender_2016', 'deathpen_2016', 'deathpenfreq_2016', 'police_threat_2016', 'univhealthcov_2016', 'healthreformbill_2016', 'envwarm_2016', 'envpoll2_2016', 'affirmact_gen_2016', 'taxdoug_2016', 'govt_reg_2016', 'gvmt_involment_2016', 'tradepolicy_2016', 'free_trade_1_2016', 'free_trade_2_2016', 'free_trade_3_2016', 'free_trade_4_2016']
    #print(len(columns))
    #print(df.shape)
    #df_clean.columns = columns
    #print(df_clean.head())
    df.columns = columns
    data_x = pd.get_dummies(df)
    label = transformLabels(labels)
    return label, data_x

def transformLabels(temp):
    label = []
    for name in temp:
        if name == "Hillary Clinton":
            label.append(1)
        elif name == "Donald Trump":
            label.append(2)
        else:
            label.append(0)
    return label


    




# Tree Model

In [55]:

params = {"max_depth": [5,10,15,20, 25], "min_samples_leaf": [5,10,15,20], "max_features": list(range(20, 160, 20))}

tree = DecisionTreeClassifier()
grid_search = GridSearchCV(tree, params, cv=5, scoring='accuracy')
grid_search.fit(data_x, label)
start = time.time()
nested_score = cross_val_score(grid_search, data_x, label, cv=10)
end = time.time()
print("Accuracy:", nested_score.mean()*100)
print("time : ", end - start)



Accuracy: 81.99723218987668
time :  21.41360902786255


# Naive Bayes (NB) & Evaluation Metrics

In [53]:
clf = GaussianNB()
clf.fit(data_x, label)

nested_score = cross_val_score(clf, data_x, label, cv=10)
print("Accuracy:", nested_score.mean()*100)


Accuracy: 80.31799199049637


# k-Nearest Neighbor (KNN)

In [56]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

knn = KNeighborsClassifier(n_neighbors=7)
pipe = Pipeline(steps=[("knn", knn)])

param_grid = {
    'knn__n_neighbors': list(range(1, 25))
}

grid_search = GridSearchCV(pipe, param_grid, cv=10)
grid_search.fit(data_x,label)
start = time.time()
nested_score = cross_val_score(grid_search, data_x, label, cv=10)
end = time.time()
print("Accuracy:", nested_score.mean()*100)
print("time : ", end - start)

Accuracy: 85.72400891577774
time :  122.71014499664307


# NN

In [41]:
from sklearn.neural_network import MLPClassifier


nn = MLPClassifier()
param_grid = {
    'nn__hidden_layer_sizes': list(range(30, 100, 10)),
    'nn__activation':['logical', 'tanh', 'relu']
}

pipe = Pipeline([("nn", nn)])

grid = GridSearchCV(pipe, param_grid, cv=5, scoring ='accuracy')
grid.fit(data_x, label)
nested_score = cross_val_score(grid, data_x, label, cv=5)
print("Accuracy:", nested_score.mean()*100)

/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 673, in fit
    return self._fit(X, y, incremental=False)
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 357, in _fit
    self._validate_hyperparameters()
  File "/Users/coleh

/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 673, in fit
    return self._fit(X, y, incremental=False)
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 357, in _fit
    self._validate_hyperparameters()
  File "/Users/coleh

/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Use

/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Use

/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [     nan      nan      nan      nan      nan      nan      nan 0.785375
 0.784125 0.789125 0.79975  0.8005   0.7965   0.801375 0.777625 0.77375
 0.781875 0.78425  0.782625 0.79025  0.787625]
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning:

...

/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 673, in fit
    return self._fit(X, y, incremental=False)
  File "/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 357, in _fit
    self._validate_hyperparameters()
  File "/Users/coleh

/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Use

/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Use

Accuracy: 80.0875


/Users/coleharper/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


# Random Forests

In [42]:
# your code goes here
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier()
param_grid = {
    'max_depth': list(range(30, 60, 5)),
    'min_samples_leaf':[8,10,12, 14, 16],
    'max_features':['sqrt','log2']
}

grid = GridSearchCV(forest, param_grid, cv=5, scoring ='accuracy')
grid.fit(data_x, label)
nested_score = cross_val_score(grid, data_x, label, cv=5)
print("Accuracy:", nested_score.mean()*100)


Accuracy: 82.75


In [56]:
import pickle

forest = RandomForestClassifier()
param_grid = {
    'max_depth': list(range(30, 60, 5)),
    'min_samples_leaf':[8,10,12, 14, 16],
    'max_features':['sqrt','log2']
}

grid = GridSearchCV(forest, param_grid, cv=5, scoring ='accuracy')
grid.fit(data_x, label)

final_model = grid
filename = 'finalized_model.sav'
pickle.dump(final_model, open(filename, 'wb'))


In [115]:
from sklearn.preprocessing import MultiLabelBinarizer
#columns = ['Crime', 'The Economy', 'Immigration', 'The Enviroment', 'Religious Liberty', 'Terrorism', 'Gay Rights', 'Education', 'Family and Medical Leave', 'Healthcare', 'Money in Politics', 'Climate Change', 'Social Security', 'Infrastructure Investment', 'Jobs', 'The Budget Deficit', 'Poverty', 'Taxes', 'Medicare', 'Abortion', 'Size of Goverment', 'Race Equality', 'Gender Equality', 'immi_contribution_2016', 'immi_naturalize_2016', 'immi_makedifficult_2016', 'immi_muslim_2016', 'abortview3_2016', 'gaymar_2016', 'view_transgender_2016', 'deathpen_2016', 'deathpenfreq_2016', 'police_threat_2016', 'univhealthcov_2016', 'healthreformbill_2016', 'envwarm_2016', 'envpoll2_2016', 'affirmact_gen_2016', 'taxdoug_2016', 'govt_reg_2016', 'gvmt_involment_2016', 'tradepolicy_2016', 'free_trade_1_2016', 'free_trade_2_2016', 'free_trade_3_2016', 'free_trade_4_2016']
record = ['Somewhat important', 'Very important', 'Not very important', 'Very important', 'Unimportant', 'Not very important', 'Somewhat important', 'Very important', 'Not very important', 'Very important', 'Very important', 'Very important', 'Not very important', 'Somewhat important', 'Very important', 'Not very important', 'Very important', 'Very important', 'Not very important', 'Very important', 'Not very important', 'Somewhat important', 'Somewhat important', 'Mostly make a contribution', 'Favor', 'Slightly easier', 'Strongly oppose', 'Legal in all cases', 'Favor', 'Should be allow to us the restrooms of the gender with which they currently identify', 'Opposed to the death penalty', 'Too often', 'Part of a broader pattern', 'Yes', 'Expanded', 'Definitely is happening', 'Pollution from human activities', 'Favor', 'Yes', 'Too little', 'We need a strong government to handle today\'s complex economic problems', 'Favor', 'Increase', 'Increase', 'Decrease', 'Increase']
#print(len(record))
#columns = ['imiss_a_2016', 'imiss_b_2016', 'imiss_c_2016', 'imiss_d_2016', 'imiss_e_2016', 'imiss_f_2016', 'imiss_g_2016', 'imiss_h_2016', 'imiss_i_2016', 'imiss_j_2016', 'imiss_k_2016', 'imiss_l_2016', 'imiss_m_2016', 'imiss_n_2016', 'imiss_o_2016','imiss_p_2016', 'imiss_q_2016','imiss_r_2016','imiss_s_2016', 'imiss_t_2016','imiss_u_2016','imiss_x_2016','imiss_y_2016', 'immi_contribution_2016', 'immi_naturalize_2016', 'immi_makedifficult_2016', 'immi_muslim_2016', 'abortview3_2016', 'gaymar_2016', 'view_transgender_2016', 'deathpen_2016', 'deathpenfreq_2016', 'police_threat_2016', 'univhealthcov_2016', 'healthreformbill_2016', 'envwarm_2016', 'envpoll2_2016', 'affirmact_gen_2016', 'taxdoug_2016', 'govt_reg_2016', 'gvmt_involment_2016', 'tradepolicy_2016', 'free_trade_1_2016', 'free_trade_2_2016', 'free_trade_3_2016', 'free_trade_4_2016']
#print(len(columns))
#df = pd.DataFrame(data=data, columns=columns)
#columns = ['Crime', 'The Economy', 'Immigration', 'The Enviroment', 'Religious Liberty', 'Terrorism', 'Gay Rights', 'Education', 'Family and Medical Leave', 'Healthcare', 'Money in Politics', 'Climate Change', 'Social Security', 'Infrastructure Investment', 'Jobs', 'The Budget Deficit', 'Poverty', 'Taxes', 'Medicare', 'Abortion', 'Size of Goverment', 'Race Equality', 'Gender Equality', 'immi_contribution_2016', 'immi_naturalize_2016', 'immi_makedifficult_2016', 'immi_muslim_2016', 'abortview3_2016', 'gaymar_2016', 'view_transgender_2016', 'deathpen_2016', 'deathpenfreq_2016', 'police_threat_2016', 'univhealthcov_2016', 'healthreformbill_2016', 'envwarm_2016', 'envpoll2_2016', 'affirmact_gen_2016', 'taxdoug_2016', 'govt_reg_2016', 'gvmt_involment_2016', 'tradepolicy_2016', 'free_trade_1_2016', 'free_trade_2_2016', 'free_trade_3_2016', 'free_trade_4_2016']
#df.columns = columns



print(res)
#dummy = pd.get_dummies(record, columns=)

print(dummy)




#record = dummy.loc[0]

#print(len(record))
#print(len(data_x.loc[1]))

record = [record]



#record = ['Somewhat important', 'Very important', 'Not very important', 'Very important', 'Unimportant', 'Not very important', 'Somewhat important', 'Very important', 'Not very important', 'Very important', 'Very important', 'Very important', 'Not very important', 'Somewhat important', 'Very important', 'Not very important', 'Very important', 'Very important', 'Not very important', 'Very important', 'Not very important', 'Somewhat important', 'Somewhat important', 'Mostly make a contribution', 'Favor', 'Slightly easier', 'Strongly oppose', 'Legal in all cases', 'Favor', 'Should be allowed to us the restrooms of the gender with which they currently identify', 'Opposed to the death penalty', 'Too often', 'Part of a broader pattern', 'Yes', 'Expanded', 'Definitely is happening', 'Pollution from humans activities', 'Favor', 'Yes', 'Too little', 'We need a strong government to handle today\'s complex economic problems', 'Favor', 'Increase', 'Increase', 'Decrease', 'Increase']
#record = ['Somewhat important', 'Very important', 'Not very important', 'Very important', 'Unimportant', 'Not very important', 'Somewhat important', 'Very important', 'Not very important', 'Very important', 'Very important', 'Very important', 'Not very important', 'Somewhat important', 'Very important', 'Not very important', 'Very important', 'Very important', 'Not very important', 'Very important', 'Not very important', 'Somewhat important', 'Somewhat important', 'Mostly make a contribution', 'Favor', 'Slightly easier', 'Strongly oppose', 'Legal in all cases', 'Favor', 'Should be allowed to us the restrooms of the gender with which they currently identify', 'Opposed to the death penalty', 'Too often', 'Part of a broader pattern', 'Yes', 'Expanded', 'Definitely is happening', 'Pollution from humans activities', 'Favor', 'Yes', 'Too little', 'We need a strong government to handle today\'s complex economic problems', 'Favor', 'Increase', 'Increase', 'Decrease', 'Increase']


loaded_model = pickle.load(open(filename, 'rb'))

# your code goes here
predict = loaded_model.predict(record)

print(predict)




ValueError: Shape of passed values is (46, 40), indices imply (46, 173)